What kind of data structures does pyspark.mllib built-in library support in Spark?
- pyspark.mllib is the builtin library for RDD-based API.

- Import pyspark.mllib recommendation submodule and Alternating Least Squares class.
- Import pyspark.mllib classification submodule and Logistic Regression with LBFGS class.
- Import pyspark.mllib clustering submodule and kmeans class.

In [1]:
# # Import the library for ALS
# from pyspark.mllib.recommendation import ALS

# # Import the library for Logistic Regression
# from pyspark.mllib.classification import LogisticRegressionWithLBFGS

# # Import the library for Kmeans
# from pyspark.mllib.clustering import KMeans

- Load the ratings.csv dataset into an RDD.
- Split the RDD using , as a delimiter.
- For each line of the RDD, using Rating() class create a tuple of userID, productID, rating.
- Randomly split the data into training data and test data (0.8 and 0.2).

In [1]:
# # Load the data into RDD
# data = sc.textFile(file_path)

# # Split the RDD 
# ratings = data.map(lambda l: l.split(','))

# # Transform the ratings RDD 
# ratings_final = ratings.map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])))

# # Split the data into training and test
# training_data, test_data = ratings_final.randomSplit([0.8, 0.2])

- Train ALS algorithm with training data and configured parameters (rank = 10 and iterations = 10).
- Drop the rating column in the test data.
- Test the model by predicting the rating from the test data.
- Return a list of two rows of the predicted ratings.

In [2]:
# # Create the ALS model on the training data
# model = ALS.train(training_data, rank=10, iterations=10)

# # Drop the ratings column 
# testdata_no_rating = test_data.map(lambda p: (p[0], p[1]))

# # Predict the model  
# predictions = model.predictAll(testdata_no_rating)

# # Return the first 2 rows of the RDD
# predictions.take(2)

- Organize ratings RDD to make ((user, product), rating).
- Organize predictions RDD to make ((user, product), rating).
- Join the prediction RDD with the ratings RDD.
- Evaluate the model using MSE between original rating and predicted rating and print it.

In [3]:
# # Prepare ratings data
# rates = ratings_final.map(lambda r: ((r[0], r[1]), r[2]))

# # Prepare predictions data
# preds = predictions.map(lambda r: ((r[0], r[1]), r[2]))

# # Join the ratings data with predictions data
# rates_and_preds = rates.join(preds)

# # Calculate and print MSE
# MSE = rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
# print("Mean Squared Error of the model for the test data = {:.2f}".format(MSE))

- Create two RDDS, one for 'spam' and one for 'non-spam (ham)'.
- Split each email in 'spam' and 'non-spam' RDDs into words.
- Print the first element in the split RDD of both 'spam' and 'non-spam'.

In [4]:
# # Load the datasets into RDDs
# spam_rdd = sc.textFile(file_path_spam)
# non_spam_rdd = sc.textFile(file_path_non_spam)

# # Split the email messages into words
# spam_words = spam_rdd.flatMap(lambda email: email.split(' '))
# non_spam_words = non_spam_rdd.flatMap(lambda email: email.split(' '))

# # Print the first element in the split RDD
# print("The first element in spam_words is", spam_words.first())
# print("The first element in non_spam_words is", non_spam_words.first())

- Create a HashingTF() instance to map email text to vectors of 200 features.
- Each message in 'spam' and 'non-spam' datasets are split into words, and each word is mapped to one feature.
- Label the features: 1 for spam, 0 for non-spam.
- Combine both the spam and non-spam samples into a single dataset.

In [5]:
# # Create a HashingTf instance with 200 features
# tf = HashingTF(numFeatures=200)

# # Map each word to one feature
# spam_features = tf.transform(spam_words)
# non_spam_features = tf.transform(non_spam_words)

# # Label the features: 1 for spam, 0 for non-spam
# spam_samples = spam_features.map(lambda features:LabeledPoint(1, features))
# non_spam_samples = non_spam_features.map(lambda features:LabeledPoint(0, features))

# # Combine the two datasets
# samples = spam_samples.join(non_spam_samples)

- Split the combined data into training and test datasets in 80:20 ratio.
- Train the Logistic Regression model with the training dataset.
- Create a prediction label from the trained model on the test dataset.
- Combine the labels in the test dataset with the labels in the prediction dataset.
- Calculate the accuracy of the trained model using original and predicted labels.

In [6]:
# # Split the data into training and testing
# train_samples,test_samples = samples.randomSplit([0.8, 0.2])

# # Train the model
# model = LogisticRegressionWithLBFGS.train(train_samples)

# # Create a prediction label from the test data
# predictions = model.predict(test_samples.map(lambda x: x.features))

# # Combine original labels with the predicted labels
# labels_and_preds = test_samples.map(lambda x: x.label).zip(predictions)

# # Check the accuracy of the model on the test data
# accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(test_samples.count())
# print("Model accuracy : {:.2f}".format(accuracy))

- Load the 5000_points dataset into an RDD named clusterRDD.
- Transform the clusterRDD by splitting the lines based on the tab ("\t").
- Transform the split RDD to create a list of integers for the two columns.
- Confirm that there are 5000 rows in the dataset.

In [7]:
# # Load the dataset into an RDD
# clusterRDD = sc.textFile(file_path)

# # Split the RDD based on tab
# rdd_split = clusterRDD.map(lambda x: x.split("\t"))

# # Transform the split RDD by creating a list of integers
# rdd_split_int = rdd_split.map(lambda x: [int(x[0]), int(x[1])])

# # Count the number of rows in RDD 
# print("There are {} rows in the rdd_split_int dataset".format(rdd_split_int.count()))

- Train the KMeans model with clusters from 13 to 16 and print the WSSSE for each cluster.
- Train the KMeans model again with the best k.
- Get the Cluster Centers (centroids) of KMeans model trained with the best k.

In [8]:
# from math import sqrt
# def error(point):
# center = model.centers[model.predict(point)]
# return sqrt(sum([x**2 for x in (point - center)]))

In [9]:
# # Train the model with clusters from 13 to 16 and compute WSSSE
# for clst in range(13, 17):
#     model = KMeans.train(rdd_split_int, clst, seed=1)
#     WSSSE = rdd_split_int.map(lambda point: error(point)).reduce(lambda x, y: x + y)
#     print("The cluster {} has Within Set Sum of Squared Error {}".format(clst, WSSSE))

# # Train the model again with the best k
# model = KMeans.train(rdd_split_int, k=15, seed=1)

# # Get cluster centers
# cluster_centers = model.clusterCenters

- Convert the rdd_split_int RDD to a Spark DataFrame, then to a pandas DataFrame.
- Create a pandas DataFrame from the cluster_centers list.
- Create a scatter plot from the pandas DataFrame of raw data (rdd_split_int_df_pandas) and overlay that with a scatter plot from the Pandas DataFrame of centroids (cluster_centers_pandas).

In [11]:
# # Convert rdd_split_int RDD into Spark DataFrame and then to Pandas DataFrame
# rdd_split_int_df_pandas = spark.createDataFrame(rdd_split_int, schema=["col1", "col2"]).toPandas()

# # Convert cluster_centers to a pandas DataFrame
# cluster_centers_pandas = pd.DataFrame(cluster_centers, columns=["col1", "col2"])

# # Create an overlaid scatter plot of clusters and centroids
# plt.scatter(rdd_split_int_df_pandas["col1"], rdd_split_int_df_pandas["col2"])
# plt.scatter(cluster_centers_pandas["col1"], cluster_centers_pandas["col2"], color="red", marker="x")
# plt.show()